<a href="https://colab.research.google.com/github/Tade5250/WeizActionClassifier/blob/main/DL_Coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Coursework

## Imports

In [1]:
!pip install torchvision
!pip install --upgrade torchvision
!pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [68]:
from torchvision.io import read_video, write_video
import os
import random
import torch.nn as nn
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torchvision.transforms import ToPILImage
from torchvision.transforms import functional as TVF
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import KFold
from torch.nn.utils.rnn import pad_sequence
import os
import torch
import random
from torch.utils.data import Dataset
from torchvision.transforms import ToPILImage
from torchvision.io import read_video
import torchvision.transforms as transforms
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch
import torch.nn as nn

In [69]:
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

# Training Flag

In [70]:
TRAINING = False

## Step 1: Getting the Data and Visualisation

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip /content/drive/MyDrive/dataset.zip -d /content/dataset

Archive:  /content/drive/MyDrive/dataset.zip
   creating: /content/dataset/videos/
   creating: /content/dataset/videos/run/
  inflating: /content/dataset/videos/run/daria_run.avi  
  inflating: /content/dataset/videos/run/ido_run.avi  
  inflating: /content/dataset/videos/run/lena_run1.avi  
  inflating: /content/dataset/videos/run/denis_run.avi  
  inflating: /content/dataset/videos/run/eli_run.avi  
  inflating: /content/dataset/videos/run/moshe_run.avi  
  inflating: /content/dataset/videos/run/lena_run2.avi  
  inflating: /content/dataset/videos/run/lyova_run.avi  
  inflating: /content/dataset/videos/run/ira_run.avi  
  inflating: /content/dataset/videos/run/shahar_run.avi  
   creating: /content/dataset/videos/wave1/
  inflating: /content/dataset/videos/wave1/ira_wave1.avi  
  inflating: /content/dataset/videos/wave1/ido_wave1.avi  
  inflating: /content/dataset/videos/wave1/lena_wave1.avi  
  inflating: /content/dataset/videos/wave1/lyova_wave1.avi  
  inflating: /content/datas

## Step 2: Process the Dataset

In [71]:
classes = {
    0: 'walk',
    1: 'run',
    2: 'jump',
    3: 'side',
    4: 'bend',
    5: 'wave1',
    6: 'wave2',
    7: 'pjump',
    8: 'jack',
    9: 'skip'
}

# Reverse mapping for class_to_idx
class_to_idx = {v: k for k, v in classes.items()}

In [72]:
class VideoDataset(Dataset):
    def __init__(self, root_dir, transform=None, min_seq_length=10, max_seq_length=50):
        self.root_dir = root_dir
        self.transform = transform
        self.min_seq_length = min_seq_length
        self.max_seq_length = max_seq_length
        self.classes = classes
        self.video_files = []
        self.class_to_idx = class_to_idx

        subdirs = sorted(os.listdir(root_dir))
        for cls_name in subdirs:
            if cls_name not in self.class_to_idx:
                raise ValueError(f"Unexpected class directory: {cls_name}. Expected one of {list(self.class_to_idx.keys())}")

        for class_label in self.class_to_idx.keys():
            class_dir = os.path.join(root_dir, class_label)
            if not os.path.exists(class_dir):
                raise ValueError(f"Class directory not found: {class_dir}")
            for video_file in os.listdir(class_dir):
                if video_file.endswith('.avi'):
                    self.video_files.append((os.path.join(class_dir, video_file), class_label))

    def __len__(self):
        return len(self.video_files)

    def __getitem__(self, idx):
        video_path, class_label = self.video_files[idx]
        video_avi, _, fps = read_video(video_path, pts_unit='sec')

        total_frames = video_avi.shape[0]

        # Sample a random sequence length
        seq_length = random.randint(self.min_seq_length, self.max_seq_length)
        if total_frames > seq_length:
            start_idx = random.randint(0, total_frames - seq_length)
            subsequence = video_avi[start_idx:start_idx + seq_length]
        else:
            start_idx = 0
            seq_length = total_frames
            subsequence = video_avi

        subsequence = subsequence.permute(0, 3, 1, 2).float() / 255.0

        # Apply transforms
        augmented = False
        if self.transform:
            augmented = True
            subsequence = self.transform(subsequence)

        metadata = {
            'class_label': class_label,
            'class_index': self.class_to_idx[class_label],
            'original_length': total_frames,
            'subsequence_length': seq_length,
            'start_index': start_idx,
            'augmented': augmented
        }

        #print("Metadata:", metadata)

        return subsequence, torch.tensor(self.class_to_idx[class_label]), metadata

class TransformSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, idx):
        x, y, metadata = self.subset[idx]
        if self.transform:
            x = self.transform(x)
            metadata['applied_transforms'] = str(self.transform)
        return x, y, metadata

    def __len__(self):
        return len(self.subset)

## Step 3: Split the Data into Train, Validate and Test Set

In [73]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):

    videos, labels, metadata = zip(*batch)

    lengths = [vid.shape[0] for vid in videos]
    max_length = max(lengths)

    padded_videos = []
    for vid in videos:
        l = vid.shape[0]
        if l < max_length:
            pad = torch.zeros((max_length - l, *vid.shape[1:]), dtype=vid.dtype)
            vid = torch.cat([vid, pad], dim=0)
        padded_videos.append(vid)

    videos_tensor = torch.stack(padded_videos, dim=0)

    labels_tensor = torch.tensor(labels)
    lengths_tensor = torch.tensor(lengths)

    return videos_tensor, labels_tensor, lengths_tensor, metadata

def random_crop(video, crop_size):
    h, w = video.shape[2], video.shape[3]
    th, tw = crop_size
    i = random.randint(0, h - th)
    j = random.randint(0, w - tw)
    return video[:, :, i:i+th, j:j+tw]

train_transform = transforms.Compose([
    transforms.Lambda(lambda x: random_crop(x, (128, 128))),
])

val_test_transform = transforms.Compose([
    transforms.Lambda(lambda x: random_crop(x, (128, 128))),
    ])

In [74]:
root_dir = '/content/dataset/videos'
base_dataset = VideoDataset(root_dir=root_dir, transform=train_transform)

labels = [base_dataset[i][1] for i in range(len(base_dataset))]

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
train_val_idx, test_idx = next(sss.split(range(len(base_dataset)), labels))

sss_val = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(sss_val.split(train_val_idx, [labels[i] for i in train_val_idx]))

train_dataset = TransformSubset(Subset(base_dataset, train_idx), train_transform)
val_dataset = TransformSubset(Subset(base_dataset, val_idx), val_test_transform)
test_dataset = TransformSubset(Subset(base_dataset, test_idx), val_test_transform)

batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Training set size: 66
Validation set size: 17
Test set size: 10


In [75]:
for batch in train_loader:
    videos, labels, lengths, metadata = batch
    print("Train videos shape:", videos.shape) # [batch_size, max_seq_length, C, H, W]
    print("Train labels shape:", labels.shape) # [batch_size]
    print("Train lengths:", lengths)

Train videos shape: torch.Size([8, 48, 3, 128, 128])
Train labels shape: torch.Size([8])
Train lengths: tensor([29, 43, 25, 24, 28, 16, 48, 25])
Train videos shape: torch.Size([8, 49, 3, 128, 128])
Train labels shape: torch.Size([8])
Train lengths: tensor([24, 30, 28, 49, 21, 33, 41, 39])
Train videos shape: torch.Size([8, 42, 3, 128, 128])
Train labels shape: torch.Size([8])
Train lengths: tensor([31, 19, 24, 22, 10, 42, 41, 40])
Train videos shape: torch.Size([8, 48, 3, 128, 128])
Train labels shape: torch.Size([8])
Train lengths: tensor([25, 17, 38, 31, 47, 35, 48, 41])
Train videos shape: torch.Size([8, 48, 3, 128, 128])
Train labels shape: torch.Size([8])
Train lengths: tensor([15, 48, 30, 23, 18, 48, 23, 17])
Train videos shape: torch.Size([8, 48, 3, 128, 128])
Train labels shape: torch.Size([8])
Train lengths: tensor([27, 38, 44, 42, 48, 48, 32, 14])
Train videos shape: torch.Size([8, 36, 3, 128, 128])
Train labels shape: torch.Size([8])
Train lengths: tensor([16, 31, 36, 24, 28

### Step 3.2: View the augmentations

In [26]:
def show_frames(video_tensor, metadata):
    label_info = f"Label: {metadata['class_index']} ({metadata['class_label']})"
    seq_info = (f"Original length: {metadata['original_length']} frames, "
                f"Subsequence: {metadata['subsequence_length']} frames (start idx: {metadata['start_index']}), "
                f"augmented: {metadata['augmented']}")
    transforms_info = f"Transforms: {metadata['applied_transforms']}"
    title = f"{label_info}\n{seq_info}\n{transforms_info}"

    video_tensor = (video_tensor.permute(0, 2, 3, 1)) * 255.0
    video_tensor = video_tensor.clamp(0, 255)

    num_frames = video_tensor.shape[0]
    cols = 5
    rows = (num_frames // cols) + (1 if num_frames % cols != 0 else 0)

    fig, axes = plt.subplots(rows, cols, figsize=(15, 3 * rows))
    fig.suptitle(title)

    axes = axes.ravel() if rows > 1 else [axes]
    for i in range(num_frames):
        ax = axes[i]
        ax.imshow(video_tensor[i].byte().numpy())
        ax.axis('off')
    # Hide any extra subplots
    for i in range(num_frames, rows * cols):
        axes[i].axis('off')
    plt.tight_layout()
    plt.show()

In [27]:
train_sample, train_label, train_metadata = train_dataset[1]

# # Visualize a few samples from the training set:
# for i, (sample, label, metadata) in enumerate(train_dataset):
#         if i < 3:
#           continue
#         if i < 6:
#           show_frames(sample, metadata)
#         else:
#           break



In [28]:
print("Training sample shape:", train_sample.shape)
print("Training sample min value:", train_sample.min().item())
print("Training sample max value:", train_sample.max().item())

Training sample shape: torch.Size([23, 3, 128, 128])
Training sample min value: 0.0235294122248888
Training sample max value: 0.6823529601097107


## Step 4: ConvLSTM model

### Step 4.1: Construct the Model

In [76]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels)
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.conv(x) + self.shortcut(x))


class CNNFeatureExtractor(nn.Module):
    def __init__(self):
      super().__init__()

      self.conv1 = nn.Sequential(
          nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False),
          nn.BatchNorm2d(64),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

      # Residual blocks
      self.conv2 = self._make_residual_layer(64, 128, stride=2)
      self.conv3 = self._make_residual_layer(128, 256, stride=2)
      self.conv4 = self._make_residual_layer(256, 512, stride=2)
      self.conv5 = self._make_residual_layer(512, 512, stride=2)

      self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

    def _make_residual_layer(self, in_channels, out_channels, stride):
      return nn.Sequential(
          ResidualBlock(in_channels, out_channels, stride),
          ResidualBlock(out_channels, out_channels, 1)
          )

    def forward(self, x):
      x = self.conv1(x)
      x = self.conv2(x)
      x = self.conv3(x)
      x = self.conv4(x)
      x = self.conv5(x)
      x = self.global_avg_pool(x)
      return x



In [77]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
            )

    def forward(self, lstm_outputs):
        attn_scores = self.attn(lstm_outputs)
        attn_weights = torch.softmax(attn_scores, dim=1)
        context = torch.sum(lstm_outputs * attn_weights, dim=1)
        return context

class CNNLSTM(nn.Module):
    def __init__(self, num_classes=10, hidden_size=256, num_layers=1, bidirectional=True, dropout_p=0.5):
        super().__init__()
        self.num_directions = 2 if bidirectional else 1

        self.cnn = CNNFeatureExtractor()

        self.lstm = nn.LSTM(input_size=512, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True,
                            bidirectional=bidirectional,
                            dropout=dropout_p if num_layers > 1 else 0)

        self.attention = Attention(hidden_size * self.num_directions)

        self.dropout = nn.Dropout(dropout_p)

        # Classification
        self.fc = nn.Linear(hidden_size * self.num_directions, num_classes)

    def forward(self, x):
        B, T, C, H, W = x.size()

        x = x.view(B * T, C, H, W)
        features = self.cnn(x)
        features = features.view(B, T, 512)


        lstm_out, (h_n, c_n) = self.lstm(features)
        context = self.attention(lstm_out)

        context = self.dropout(context)
        out = self.fc(context)
        return out


### Step 4.2: Train the Model

In [78]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNLSTM(num_classes=10, hidden_size=256, num_layers=1).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

early_stopping_patience = 7
best_val_loss = float('inf')
early_stopping_counter = 0
num_epochs = 100

def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for videos, labels, lengths, metadata in loader:
        videos = videos.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(videos)
        loss = criterion(outputs, labels)
        loss.backward()


        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        running_loss += loss.item() * videos.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def validate_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (videos, labels, lengths, metadata) in enumerate(loader):
            videos = videos.to(device)
            labels = labels.to(device)
            outputs = model(videos)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * videos.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Print predictions and labels for the current batch
            # print(f"Batch {batch_idx + 1}:")
            # print("Predictions:", predicted.cpu().numpy())
            # print("Labels:     ", labels.cpu().numpy())

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# Main training loop
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []
best_model_wts = None

for epoch in range(num_epochs):
    print(f"\nEpoch [{epoch+1}/{num_epochs}]")

    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # Validate
    val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    scheduler.step(val_loss)


    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0
        torch.save(best_model_wts, 'CNN_LSTM.pth')
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered!")
            break

print("Training complete.")

if best_model_wts is not None:
    model.load_state_dict(torch.load('CNN_LSTM.pth'))

test_loss, test_acc = validate_epoch(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")



/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



Epoch [1/100]
Train Loss: 2.3008, Train Acc: 0.0606 | Val Loss: 2.3344, Val Acc: 0.0000

Epoch [2/100]
Train Loss: 2.2266, Train Acc: 0.0909 | Val Loss: 2.3975, Val Acc: 0.0588

Epoch [3/100]
Train Loss: 2.1551, Train Acc: 0.2273 | Val Loss: 2.4575, Val Acc: 0.0588

Epoch [4/100]
Train Loss: 1.9751, Train Acc: 0.3636 | Val Loss: 2.4710, Val Acc: 0.0000

Epoch [5/100]
Train Loss: 1.7889, Train Acc: 0.4394 | Val Loss: 2.4698, Val Acc: 0.0588

Epoch [6/100]
Train Loss: 1.6530, Train Acc: 0.4091 | Val Loss: 2.2719, Val Acc: 0.1176

Epoch [7/100]
Train Loss: 1.5869, Train Acc: 0.3485 | Val Loss: 1.9532, Val Acc: 0.1176

Epoch [8/100]
Train Loss: 1.4525, Train Acc: 0.4545 | Val Loss: 1.9920, Val Acc: 0.1765

Epoch [9/100]
Train Loss: 1.3946, Train Acc: 0.4697 | Val Loss: 2.0449, Val Acc: 0.2353

Epoch [10/100]
Train Loss: 1.2729, Train Acc: 0.4697 | Val Loss: 1.8299, Val Acc: 0.2941

Epoch [11/100]
Train Loss: 1.2547, Train Acc: 0.4848 | Val Loss: 1.7798, Val Acc: 0.1765

Epoch [12/100]
Tra

### Step 4.3: Test Best Model

### Step 4.3: Save the Model

## Step 5: Loading the Pretrained model

### Step 5.1: Apply model to Test Data

### Step 5.2: Analyse and Display the Results and Accuracy

# Report

**Student Number: 2619191a**